In [1]:
import numpy as np
import xarray as xr
import datetime as dt
import os
import matplotlib.pyplot as plt

In [2]:
# Directories '/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/NMME_SST/by_lead'
basepath='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW'
dir_hc = basepath+'/hindcast'#/NMME_SST/by_lead
#'/Volumes/Data_2TB/NMME/SST/by_lead/hindcast';
dir_fc = basepath+'/forecast'#/NMME_SST/by_lead
#'/Volumes/Data_2TB/NMME/SST/by_lead/forecast';
dir_out = basepath+'/concatenated'#/NMME_SST/by_lead
#'/Volumes/Data_2TB/NMME/SST/by_lead/concatenated';

In [3]:
# Model names
#           1              2               3            4              5             6
mods = ['CanCM4i', 'COLA-RSMAS-CCSM4', 'GEM-NEMO', 'GFDL-SPEAR', 'NASA-GEOSS2S', 'NCEP-CFSv2']
nmod = len(mods)
nl = dict(zip(mods,[11, 11, 11, 11, 8, 9])) # Max lead time for each model
nl

{'CanCM4i': 11,
 'COLA-RSMAS-CCSM4': 11,
 'GEM-NEMO': 11,
 'GFDL-SPEAR': 11,
 'NASA-GEOSS2S': 8,
 'NCEP-CFSv2': 9}

In [4]:
# Loop through models
print('\nConcatenating hindcasts and forecasts...\n')
for modi in [ 'NCEP-CFSv2',]:#mods: # for imod = 1:nmod
    
    print(f'\nProcessing {modi}...\n')
    print(' Lead')
    
    # Loop through lead times
    for il in (8,):#range(0,nl[modi]):
        print(' ',il)
        
        # Load hindcast
        f_inh = f'{dir_hc}/sst_{modi}_hindcast_l{il}.nc';
        with xr.open_dataset(f_inh,decode_times=False) as fih:
            lat=fih.lat.values
            lon=fih.lon.values
            time1=fih.time.values
            sst1 = fih.sst.values
        if not modi=='NCEP-CFSv2': #if imod~=6
            sst1 = fih.sst.values
            nm = np.shape(sst1)[1] # of ensemble members
        else:
            nm = 24; # For CFSv2
            sst1 = fih.sst[:,:24,:,:].values

        # Get mask
        #if il==0:
        tmp = sst1[0,0,:,:] #sst1(:,:,1,1);
        mask = np.ones(np.shape(tmp))
        if modi in {'CanCM4i', 'GEM-NEMO'}:
            mask[tmp==0] = 0
        elif modi in {'COLA-RSMAS-CCSM4', 'GFDL-SPEAR'}:
            mask[np.abs(tmp)>1e3] = 0
        elif modi in {'NASA-GEOSS2S', 'NCEP-CFSv2'}:
            mask[np.isnan(tmp)] = 0

        # Concatenate
        if not modi in {'COLA-RSMAS-CCSM4','NCEP-CFSv2'}:#imod~=2 && imod~=6
            # Load forecast
            f_inf = f'{dir_fc}/sst_{modi}_forecast_l{il}.nc'
            with xr.open_dataset(f_inf,decode_times=False) as fif:
                sst2=fif.sst.values
                time2=fif.time.values
            
            # Constrain to number of members in hindcast
            sst2 = sst2[:,:nm,:,:]
           
            # Concatenate hindcast and forecast, eliminating temporal overlap
            ind = time2>np.max(time1) #find(time2>max(time1));
            time2 = time2[ind];
            sst2 = sst2[ind,:,:,:]
            time = np.concatenate([time1,time2])
            sst = np.concatenate((sst1,sst2),axis=0);
        else:
            sst = sst1;
            time = time1;
            
        # Adjust units if necessary
        if modi in {'CanCM4i', 'GEM-NEMO'}:
            sst = sst - 273.15 # Convert K to C
        
        # Remove outliers / missing data and apply mask from hindcast
        sst=np.where((np.abs(sst)>100) | (mask*np.ones(np.shape(sst))==0),np.nan,sst) #sst(abs(sst)>100) = nan;

        # Save to file
        f_out = f'{dir_out}/sst_{modi}_l{il}_concatenated.nc'
        with xr.open_dataset(f_inh,decode_times=False) as fi:
            xout=xr.Dataset(data_vars={'lon':(['X',],lon),
                                    'lat':(['Y',],lat),
                                    'time':(['S'],time),
                                    'sst':(['S','M','Y','X'],sst)},
                        coords=dict(X=fi.X,Y=fi.Y,M=("M",np.arange(0,nm)),S=("S",time)),)
            xout.to_netcdf(f_out,mode='w')
        del sst; del sst1;
print('\nDone\n\n')


Concatenating hindcasts and forecasts...


Processing NCEP-CFSv2...

 Lead
  7

Done


